In [1]:
import os
import sys

import lxml
import urllib.request     # available?
import requests
import datetime

from bs4 import BeautifulSoup
from urllib.parse import urlencode, quote

In [2]:
datetime_format = datetime.datetime.strptime('2019-01-02 23:00:00', '%Y-%m-%d %H:%M:%S')
datetime.datetime.strftime(datetime_format, '%Y.%m/%d(%a)')

'2019.01/02(Wed)'

In [3]:
# 현재 화일이 있는 working dir.
root_name = 'web_beautifulsoup_scrapping'

dir_work = os.getcwd()

ROOT = os.path.join(dir_work, root_name, '')
dir_static_result = os.path.join(ROOT, *['static', 'result', '',])
dir_static_result

'C:\\Users\\nitt0\\Documents\\GitHub\\web_beautifulsoup_scrapping\\jupyter_nb\\web_beautifulsoup_scrapping\\static\\result\\'

In [4]:
# 이렇게 할 필요는 없었다.
KEY_WORD = '미세먼지'

TARGET_URL_BEFORE_PAGE_NUM = "http://news.donga.com/search?p="
TARGET_URL_BEFORE_KEWORD = '&query='
TARGET_URL_REST = '&check_news=1&more=1&sorting=3&search_date=1&v1=&v2=&range=3'

OUT_F_NAME = '_1_thadd_article.pdb'

# full-queried 타겟URL 작성 (키워드 포함)

target_URL = TARGET_URL_BEFORE_PAGE_NUM \
    + TARGET_URL_BEFORE_KEWORD \
    + quote(KEY_WORD) \
    + TARGET_URL_REST
target_URL

'http://news.donga.com/search?p=&query=%EB%AF%B8%EC%84%B8%EB%A8%BC%EC%A7%80&check_news=1&more=1&sorting=3&search_date=1&v1=&v2=&range=3'

In [5]:
url = 'http://news.donga.com/search'
param_dict = {
    'p' : '',
    'query' : KEY_WORD,
    'check_news' : '1',
    'more' : '1',
    'sorting' : '1',         # 정확도=3, 최신순=1
    'search_date' : '1',
    'v1' : '',
    'v2' : '',
    'range' : '3',  
}


url_all = url + '?' + urlencode(query=param_dict)
url_all

'http://news.donga.com/search?p=&query=%EB%AF%B8%EC%84%B8%EB%A8%BC%EC%A7%80&check_news=1&more=1&sorting=1&search_date=1&v1=&v2=&range=3'

In [6]:
target_URL == url_all

False

In [7]:
source_code_from_url = urllib.request.urlopen(url_all)
soup = BeautifulSoup(source_code_from_url, 'lxml', from_encoding='utf-8')

# select 나 find_all 이나 결과는 똑같다.
content_of_articles = soup.select('p.tit')
# content_of_articles

findings =  soup.find_all('p', 'tit')
# findings

In [8]:
search_md_string = title_text_string = f"## Search Result for '{KEY_WORD}'\n\n"
# title_text_string = f"## Search Result for '{KEY_WORD}'\n\n"

print(search_md_string)
for i, title in enumerate(soup.find_all('p', 'tit'),1):
    date_text = title.select('span')[-1].get_text()
    datetime_format = datetime.datetime.strptime(date_text, '%Y-%m-%d %H:%M')
    date_new = datetime.datetime.strftime(datetime_format, '%y.%m/%d(%a)')

    title_text = title.get_text()\
                    .replace('\n', '')\
                    .replace('  ', '')\
                    .replace(date_text,'')

    title_link = title.select('a')
    article_URL = title_link[0]['href']
    
    title_text_string += f"{i:02}.[{date_new}] {title_text[:35]}..\n"
    print(f"{i:02},{date_new} {title_text}\n{article_URL}\n")
    search_md_string += f"{i:02}-[{date_new}] [{title_text}]({article_URL})\n"

search_md_string += "<br><br>\n"

## Search Result for '미세먼지'


01,19.02/08(Fri) 충청권, ‘2030 아시아경기’ 공동유치 나섰다 
http://news.donga.com/3/all/20190207/94016284/1

02,19.02/08(Fri) [윤희웅의 SNS민심]전기차는 성능, 수소차는 주가에 관심 
http://news.donga.com/3/all/20190208/94016707/1

03,19.02/07(Thu) 대구시, 2022년까지 도심숲 100곳 조성 
http://news.donga.com/3/all/20190206/94000023/1

04,19.02/07(Thu) [광화문에서/동정민]“서울 하늘을 다시 파랗게”… ‘플래닛 외교’에 나설 때다 
http://news.donga.com/3/all/20190207/94000435/1

05,19.02/07(Thu) 여야 양보없는 대치-비핵화 빅이슈에… 2월 국회 ‘실종’ 위기 
http://news.donga.com/3/all/20190207/94000499/1

06,19.02/07(Thu) 지난달 최악 미세먼지, 75%가 중국 등서 왔다 
http://news.donga.com/3/all/20190206/94000355/1

07,19.02/02(Sat) “매서운 파도에 올라타라” 한겨울 동해는 ‘서핑 천국’ 
http://news.donga.com/3/all/20190202/93972275/1

08,19.02/02(Sat) [동아광장/박상준]20대 청춘남녀, 혐오의 날 내려두고 서로를 품어라 
http://news.donga.com/3/all/20190202/93972144/1

09,19.02/01(Fri) [기고/이관영]수소경제 주도권 잡아라 
http://news.donga.com/3/all/20190201/93958373/1

10,19.02/01(Fri) [독자위원회 좌담]손혜원, 권력 이용해 사익 취하려 했다면 강하게 꾸짖어야 
http://news.donga.com/3/a

In [9]:
print(title_text_string)

## Search Result for '미세먼지'

01.[19.02/08(Fri)] 충청권, ‘2030 아시아경기’ 공동유치 나섰다 ..
02.[19.02/08(Fri)] [윤희웅의 SNS민심]전기차는 성능, 수소차는 주가에 관심 ..
03.[19.02/07(Thu)] 대구시, 2022년까지 도심숲 100곳 조성 ..
04.[19.02/07(Thu)] [광화문에서/동정민]“서울 하늘을 다시 파랗게”… ‘플래닛 외교..
05.[19.02/07(Thu)] 여야 양보없는 대치-비핵화 빅이슈에… 2월 국회 ‘실종’ 위기 ..
06.[19.02/07(Thu)] 지난달 최악 미세먼지, 75%가 중국 등서 왔다 ..
07.[19.02/02(Sat)] “매서운 파도에 올라타라” 한겨울 동해는 ‘서핑 천국’ ..
08.[19.02/02(Sat)] [동아광장/박상준]20대 청춘남녀, 혐오의 날 내려두고 서로를 ..
09.[19.02/01(Fri)] [기고/이관영]수소경제 주도권 잡아라 ..
10.[19.02/01(Fri)] [독자위원회 좌담]손혜원, 권력 이용해 사익 취하려 했다면 강하..
11.[19.01/31(Thu)] [인천판/게시판]인천 건강강좌 外 ..
12.[19.01/31(Thu)] “다이어트-화장품 등 가짜 후기 단속… 생활용품 불신 해소할 것..
13.[19.01/30(Wed)] LG, 아동시설에 공기청정기 지원 ..
14.[19.01/30(Wed)] 부산 사상구 “미세먼지-악취-폐수 꼼짝 마!” ..
15.[19.01/30(Wed)] 극한 환경을 이겨낸 강인한 식물의 생명력이 피부를 지켜준다! ..



In [10]:
print(search_md_string)

## Search Result for '미세먼지'

01-[19.02/08(Fri)] [충청권, ‘2030 아시아경기’ 공동유치 나섰다 ](http://news.donga.com/3/all/20190207/94016284/1)
02-[19.02/08(Fri)] [[윤희웅의 SNS민심]전기차는 성능, 수소차는 주가에 관심 ](http://news.donga.com/3/all/20190208/94016707/1)
03-[19.02/07(Thu)] [대구시, 2022년까지 도심숲 100곳 조성 ](http://news.donga.com/3/all/20190206/94000023/1)
04-[19.02/07(Thu)] [[광화문에서/동정민]“서울 하늘을 다시 파랗게”… ‘플래닛 외교’에 나설 때다 ](http://news.donga.com/3/all/20190207/94000435/1)
05-[19.02/07(Thu)] [여야 양보없는 대치-비핵화 빅이슈에… 2월 국회 ‘실종’ 위기 ](http://news.donga.com/3/all/20190207/94000499/1)
06-[19.02/07(Thu)] [지난달 최악 미세먼지, 75%가 중국 등서 왔다 ](http://news.donga.com/3/all/20190206/94000355/1)
07-[19.02/02(Sat)] [“매서운 파도에 올라타라” 한겨울 동해는 ‘서핑 천국’ ](http://news.donga.com/3/all/20190202/93972275/1)
08-[19.02/02(Sat)] [[동아광장/박상준]20대 청춘남녀, 혐오의 날 내려두고 서로를 품어라 ](http://news.donga.com/3/all/20190202/93972144/1)
09-[19.02/01(Fri)] [[기고/이관영]수소경제 주도권 잡아라 ](http://news.donga.com/3/all/20190201/93958373/1)
10-[19.02/01(Fri)] [[독자위원회 좌담]손혜원, 권력 이용해 사익 취하려 했